# Лабораторная работа 4

Решить три задачи из задачника. 
Найти приближенные решения, применяя метод монте-карло сравнить их с правильным ответом.

## § 2. Классическое вероятностное пространство

### Задача 2.49

**Условие**

Две игральные кости бросаются $r$ раз. Найти вероятность того, что каждая из шести комбинацияй $(1,1), ..., (6,6)$ появится по меньшей мере один раз.

**Аналитическое решение**

Пусть мы бросаем две честные игральные кости. Всего возможных результатов одного броска:

$$
36 = 6 \times 6
$$

Вероятность появления одной конкретной диагональной пары за один бросок:

$$
p = \frac{1}{36}
$$

Мы ищем вероятность того, что все шесть диагональных комбинаций  
$(1,1), (2,2), \dots, (6,6)$  
появятся хотя бы один раз за $r$ бросков.

Обозначим $A_i$ — событие «комбинация $(i,i)$ не выпала ни разу» $\Rightarrow$ нужна вероятность противоположного события:

$$
P(r) = 1 - \mathbb{P}(A_1 \cup A_2 \cup \dots \cup A_6).
$$

Это *формула включений-исключений*. Используем её, т.к. она позволяет опр. в-ть события вида "все нужные события произошли хотя бы один раз" при условии, что эти события:
- зависят друг от друга;
- пересекаются;
- их невозможно посчитать напрямую через сумму.

Общее выражение для включений–исключений:
$$
P(A_1 \cup \dots \cup A_6)
= \sum_{k=1}^6 (-1)^{k+1}
\sum_{i_1 < \dots < i_k} 
P(A_{i_1} \cap \dots \cap A_{i_k}).
$$

Вероятность пересечения $k$ событий одинакова:
$$
P(A_{i_1} \cap \dots \cap A_{i_k})
= \left(1 - \frac{k}{36}\right)^r,
$$
т.к. запрещено ровно $k$ исходов из 36.

Количество таких пересечений:
$$
\binom{6}{k}.
$$

Подстановка этих двух фактов в общее выражение даёт
$$
P(A_1 \cup\dots\cup A_6)
= \sum_{k=1}^6 (-1)^{k+1}
\binom{6}{k}
\left(1 - \frac{k}{36}\right)^r.
$$

$\Rightarrow P(r) = 1 - P(A_1 \cup\dots\cup A_6)$

Формула включений–исключений вычисляет вероятность того, что отсутствует хотя бы одна комбинация, учитывая все перекрытия таких случаев:

$$
P(r) =
\sum_{k=0}^{6}
(-1)^k \binom{6}{k}
\left(1 - \frac{k}{36}\right)^r.
$$

- $\binom{6}{k}$ — число способов выбрать $k$ комбинаций из шести, которые могут не появиться.  
- $(1 - k/36)^r$ — вероятность того, что выбранные $k$ комбинаций не появятся ни разу за $r$ бросков.  
- $(-1)^k$ — корректирует пересечения: сначала добавляются одиночные случаи, затем вычитаются двойные пересечения, затем добавляются тройные и т.д.

Таким образом формула перебирает все варианты того, какие комбинации могли не выпасть, и в итоге даёт вероятность того, что не отсутствует ни одна комбинация, то есть все шесть диагональных пар появились хотя бы один раз.

**Python: аналитическое решение + Монте-Карло с разными генераторами случайных чисел**

In [ ]:
import random
import math


h_line = ("-"*55)


# Аналитическое решение
def probability_analytical(n_rolls: int) -> float:
    """
    Аналитическая вероятность того, что все диагональные комбинации
    (1,1)...(6,6) появятся хотя бы раз за n_rolls бросков двух кубиков.

    Формула включений–исключений:
    P = Σ (-1)^k * C(6,k) * (1 - k/36)^n
    """
    total = 0.0
    for k in range(7):
        term = ((-1) ** k) * math.comb(6, k) * (1 - k / 36) ** n_rolls
        total += term
    return total


# Монте-Карло - единичный эксперемент
def run_single_experiment(n_rolls: int, rng: random.Random) -> bool:
    """
    True если ВСЕ диагональные пары (i,i) встретились хотя бы раз.
    """
    seen_diagonals = set()

    for _ in range(n_rolls):
        d1 = rng.randint(1, 6)
        d2 = rng.randint(1, 6)

        if d1 == d2:
            seen_diagonals.add(d1)

        if len(seen_diagonals) == 6:
            return True  # все 6 комбинаций собраны

    return len(seen_diagonals) == 6


# Монте-Карло
def probability_monte_carlo(n_rolls: int, n_experiments: int, seed: int = 42) -> float:
    rng = random.Random(seed)
    success_count = 0

    for _ in range(n_experiments):
        if run_single_experiment(n_rolls, rng):
            success_count += 1

    return success_count / n_experiments


# таблица
def print_results_table(n_rolls_list, n_experiments=50000):
    print(h_line)
    print(f"{'r (rolls)':>10} | {'Analytical':>12} | {'Monte Carlo':>12} | {'Delta':>10}")
    print(h_line)

    for n_rolls in n_rolls_list:
        p_an = probability_analytical(n_rolls)
        p_mc = probability_monte_carlo(n_rolls, n_experiments)
        delta = abs(p_an - p_mc)

        print(f"{n_rolls:10d} | {p_an:12.6f} | {p_mc:12.6f} | {delta:10.6f}")

    print(h_line)


if __name__ == "__main__":
    # кол-во бросков r
    r_values = [50, 100, 200, 300, 400, 500]
    
    # таблица
    print_results_table(r_values, n_experiments=50000)


-------------------------------------------------------
 r (rolls) |   Analytical |  Monte Carlo |      Delta
-------------------------------------------------------
        50 |     0.174088 |     0.172020 |   0.002068
       100 |     0.687507 |     0.688760 |   0.001253
       200 |     0.978720 |     0.979380 |   0.000660
       300 |     0.998719 |     0.998660 |   0.000059
       400 |     0.999923 |     0.999840 |   0.000083
       500 |     0.999995 |     1.000000 |   0.000005
-------------------------------------------------------


**Почему ошибка Монте-Карло уменьшается при увеличении числа бросков**

Оценка вероятности методом Монте-Карло - среднее значение бернуллиевской случайной величины:

$$
\hat{P} = \frac{1}{N} \sum_{i=1}^N I_i,
$$

где  

$$
I_i =
\begin{cases}
1, & \text{если в i-м эксперименте все 6 комбинаций } (k,k) \text{ появились хотя бы раз},\\[4pt]
0, & \text{иначе}.
\end{cases}
$$

Т.к. $\hat{P}$ — среднее из $N$ независимых бернуллиевских переменных, его дисперсия равна:

$$
\mathrm{Var}(\hat{P})
= \frac{P(r)\,\bigl(1 - P(r)\bigr)}{N}.
$$

**Что происходит при увеличении количества бросков $r$?**

Аналитическая вероятность стремится к 1:

$$
P(r) \rightarrow 1
\Rightarrow
1 - P(r) \rightarrow 0
\Rightarrow
\mathrm{Var}(\hat{P}) \rightarrow 0.
$$

Когда вероятность $P(r)$ близка к единице, почти каждый эксперимент Монте-Карло заканчивается "успехом".  
Колебания оценки становятся минимальными - метод Монте-Карло почти всегда выдаёт одно и то же значение.

Поэтому **разница между аналитикой и Монте-Карло резко уменьшается при больших $r$**.


### Задача 2.15

**Условие**

Колоду карт (36 листов) наудачу разделяют на 2 равные пачки. Чему равна вероятность, что:

а) в каждой из пачек окажется 2 туза;

б) в одной из пачек окажутся все 4 туза;

в) в пачках окажется по равному числу красных карт?

**Решение**

Число способов выбрать $k$ карт из $n$ без учёта порядка:

$$
\binom{n}{k} = \frac{n!}{k!(n-k)!}
$$

При разбиении 36-карточной колоды на две равные пачки по 18 карт общее число возможных разбиений:

$$
\text{Всего разбиений} = \binom{36}{18}.
$$

Во всех пунктах вероятность вычисляется как:

$$
P = \frac{\text{число благоприятных разбиений}}{\binom{36}{18}}.
$$

В Python эти сочетания вычисляются функцией:

```python
math.comb(n, k)
```

**(a) В каждой из пачек окажется 2 туза**

В колоде 4 туза. Чтобы в обеих пачках оказалось по 2 туза, в первую пачку должно попасть ровно 2 туза.

Число способов выбрать 2 туза из 4:

$$
\binom{4}{2}.
$$

Оставшиеся 16 карт добираются из 32 нетузов:

$$
\binom{32}{16}.
$$

Таким образом, число благоприятных разбиений:

$$
\binom{4}{2}\,\binom{32}{16}.
$$

Вероятность:

$$
P_a =
\frac{
\binom{4}{2}\,\binom{32}{16}
}{
\binom{36}{18}
}.
$$

In [19]:
import random
import math

# Аналитическая формула для пункта (а)
def prob_a_analytic():
    return (math.comb(4, 2) * math.comb(32, 16)) / math.comb(36, 18)

# Монте-Карло
def prob_a_mc(n_trials=100000, seed=42):
    rng = random.Random(seed)

    # строим колоду: 4 туза + 32 нетуза
    deck = ["A"] * 4 + ["N"] * 32

    success = 0

    for _ in range(n_trials):
        rng.shuffle(deck)
        pack1 = deck[:18]
        pack2 = deck[18:]

        if pack1.count("A") == 2 and pack2.count("A") == 2:
            success += 1

    return success / n_trials

print("Пункт (а)")
print("Аналитически:", prob_a_analytic())
print("Монте-Карло :", prob_a_mc())
print("Delta =", abs(prob_a_analytic() - prob_a_mc()))

Пункт (а)
Аналитически: 0.3974025974025974
Монте-Карло : 0.3953
Delta = 0.002102597402597439


**(б) в одной из пачек окажутся все 4 туза**

Для того чтобы все тузы оказались в первой пачке, нужно выбрать все 4 туза, а оставшиеся 14 карт добрать из 32 нетузов:

$$
\binom{32}{14}.
$$

То же количество разбиений соответствует случаю, когда все тузы во второй пачке.

Поэтому число благоприятных разбиений:

$$
2\,\binom{32}{14}.
$$

Вероятность:

$$
P_b =
\frac{
2\,\binom{32}{14}
}{
\binom{36}{18}
}.
$$

In [21]:
import random
import math

# Аналитическая формула для пункта (б)
def prob_b_analytic():
    return (2 * math.comb(32, 14)) / math.comb(36, 18)

# Монте-Карло
def prob_b_mc(n_trials=100000, seed=42):
    rng = random.Random(seed)

    deck = ["A"] * 4 + ["N"] * 32

    success = 0

    for _ in range(n_trials):
        rng.shuffle(deck)
        pack1 = deck[:18]
        pack2 = deck[18:]

        if pack1.count("A") == 4 or pack2.count("A") == 4:
            success += 1

    return success / n_trials

print("Пункт (б)")
print("Аналитически:", prob_b_analytic())
print("Монте-Карло :", prob_b_mc())
print("Delta =", abs(prob_b_analytic() - prob_b_mc()))

Пункт (б)
Аналитически: 0.1038961038961039
Монте-Карло : 0.10414
Delta = 0.0002438961038960935


**(в) в пачках окажется по равному числу красных карт**

В колоде 18 красных и 18 чёрных карт. Чтобы количество красных в обеих пачках совпадало, в первую пачку должно попасть ровно 9 красных.

Число способов выбрать 9 красных из 18:

$$
\binom{18}{9}.
$$

Аналогично выбираются 9 чёрных:

$$
\binom{18}{9}.
$$

Число благоприятных разбиений:

$$
\binom{18}{9}\,\binom{18}{9}.
$$

Вероятность:

$$
P_v =
\frac{
\binom{18}{9}\,\binom{18}{9}
}{
\binom{36}{18}
}.
$$

In [20]:
import random
import math

# Аналитическая формула для пункта (в)
def prob_v_analytic():
    return (math.comb(18, 9) * math.comb(18, 9)) / math.comb(36, 18)

# Монте-Карло
def prob_v_mc(n_trials=100000, seed=42):
    rng = random.Random(seed)

    # 18 красных, 18 чёрных
    deck = ["R"] * 18 + ["B"] * 18

    success = 0

    for _ in range(n_trials):
        rng.shuffle(deck)
        pack1 = deck[:18]
        pack2 = deck[18:]

        if pack1.count("R") == pack2.count("R"):
            success += 1

    return success / n_trials

print("Пункт (в)")
print("Аналитически:", prob_v_analytic())
print("Монте-Карло :", prob_v_mc())
print("Delta =", abs(prob_v_analytic() - prob_v_mc()))

Пункт (в)
Аналитически: 0.2604814497917183
Монте-Карло : 0.26126
Delta = 0.0007785502082817142


## § 3. Геометрические вероятности

**Условие**

Точка $(\xi, \eta, \zeta)$ выбирается равномерно в кубе $[0,1]^3$. Требуется найти вероятность того, что квадратное уравнение $\xi x^2 + \eta x + \zeta = 0$ имеет действительные корни.

**Решение**

Корни квадратного уравнения действительны тогда и только тогда, когда его дискриминант неотрицателен:  
$$
\eta^2 - 4\xi \zeta \ge 0.
$$

Так как $\xi,\eta,\zeta \ge 0$, это условие равносильно  
$$
\zeta \le \frac{\eta^2}{4\xi}.
$$

Поскольку точка $(\xi,\eta,\zeta)$ выбирается равномерно в $[0,1]^3$, искомая вероятность равна объёму множества  
$$
\{(\xi, \eta, \zeta) \in [0,1]^3 : \zeta \le \frac{\eta^2}{4\xi} \}.
$$

То есть
$$
P = \int_0^1 \int_0^1 \int_0^1 \mathbf{1}\!\left(\zeta \le \frac{\eta^2}{4\xi}\right)\, d\zeta\, d\eta\, d\xi,
$$
где $\mathbf{1}(\cdot)$ — индикатор события (равен 1, если условие выполняется, и 0 иначе).

---

Для фиксированных $\xi$ и $\eta$ рассмотрим внутренний интеграл по $\zeta$:
$$
\int_0^1 \mathbf{1}\!\left(\zeta \le \frac{\eta^2}{4\xi}\right)\, d\zeta.
$$

Обозначим
$$
A(\xi,\eta) = \frac{\eta^2}{4\xi}.
$$

Тогда:

- если $A(\xi,\eta) \ge 1$, то условие $\zeta \le A(\xi,\eta)$ выполняется для всех $\zeta \in [0,1]$, и интеграл по $\zeta$ равен 1;
- если $0 \le A(\xi,\eta) < 1$, то $\zeta$ пробегает от $0$ до $A(\xi,\eta)$, и интеграл равен $A(\xi,\eta)$.

Итак,
$$
\int_0^1 \mathbf{1}\!\left(\zeta \le \frac{\eta^2}{4\xi}\right)\, d\zeta
=
\min\!\left(1,\;\frac{\eta^2}{4\xi}\right),
$$
и
$$
P = \int_0^1 \int_0^1 \min\left(1,\;\frac{\eta^2}{4\xi}\right)\, d\eta\, d\xi.
$$

---

Теперь вычислим этот двойной интеграл.

Зафиксируем $\eta \in (0,1]$ и проинтегрируем по $\xi$.  
Неравенство
$$
\frac{\eta^2}{4\xi} \ge 1
$$
эквивалентно
$$
\xi \le \frac{\eta^2}{4}.
$$

Поэтому при фиксированной $\eta$:
$$
\min\left(1,\frac{\eta^2}{4\xi}\right)
=
\begin{cases}
1, & 0 < \xi \le \frac{\eta^2}{4},\\[6pt]
\dfrac{\eta^2}{4\xi}, & \frac{\eta^2}{4} < \xi \le 1.
\end{cases}
$$

Тогда внутренний интеграл по $\xi$:
$$
\int_0^1 \min\left(1,\frac{\eta^2}{4\xi}\right)\, d\xi
=
\int_0^{\eta^2/4} 1\, d\xi
+
\int_{\eta^2/4}^1 \frac{\eta^2}{4\xi}\, d\xi.
$$

Вычислим каждую часть.

1) Первый интеграл:
$$
\int_0^{\eta^2/4} 1\, d\xi = \frac{\eta^2}{4}.
$$

2) Второй интеграл:
$$
\int_{\eta^2/4}^1 \frac{\eta^2}{4\xi}\, d\xi
=
\frac{\eta^2}{4} \int_{\eta^2/4}^1 \frac{1}{\xi}\, d\xi
=
\frac{\eta^2}{4}\,[\ln \xi]_{\xi=\eta^2/4}^{\xi=1}
=
\frac{\eta^2}{4}\,\bigl(\ln 1 - \ln(\eta^2/4)\bigr).
$$

Так как $\ln1 = 0$, имеем
$$
\int_{\eta^2/4}^1 \frac{\eta^2}{4\xi}\, d\xi
=
-\,\frac{\eta^2}{4}\,\ln\!\left(\frac{\eta^2}{4}\right).
$$

Итак, при фиксированной $\eta$:
$$
\int_0^1 \min\left(1,\frac{\eta^2}{4\xi}\right)\, d\xi
=
\frac{\eta^2}{4}
-
\frac{\eta^2}{4}\,\ln\!\left(\frac{\eta^2}{4}\right)
=
\frac{\eta^2}{4}\left[\,1 - \ln\!\left(\frac{\eta^2}{4}\right)\right].
$$

Подставим в интеграл по $\eta$:
$$
P = \int_0^1 \left[ \frac{\eta^2}{4}\left(1 - \ln\!\left(\frac{\eta^2}{4}\right)\right)\right] d\eta.
$$

Раскроем скобки:
$$
P = \frac{1}{4}\int_0^1 \eta^2\, d\eta
\;-\;
\frac{1}{4}\int_0^1 \eta^2 \ln\!\left(\frac{\eta^2}{4}\right)\, d\eta.
$$

Разберём второй интеграл.  
Раскроем логарифм:
$$
\ln\!\left(\frac{\eta^2}{4}\right)
= \ln(\eta^2) - \ln 4
= 2\ln\eta - \ln 4.
$$

Тогда
$$
\eta^2 \ln\!\left(\frac{\eta^2}{4}\right)
=
\eta^2\,(2\ln\eta - \ln 4)
=
2\eta^2\ln\eta - \eta^2\ln 4.
$$

Следовательно,
$$
\int_0^1 \eta^2 \ln\!\left(\frac{\eta^2}{4}\right)\, d\eta
=
2\int_0^1 \eta^2\ln\eta\, d\eta
-
\ln 4\int_0^1 \eta^2\, d\eta.
$$

Подставляем обратно:
$$
P = \frac{1}{4}\int_0^1 \eta^2\, d\eta
-
\frac{1}{4}\left[
2\int_0^1 \eta^2\ln\eta\, d\eta
-
\ln 4\int_0^1 \eta^2\, d\eta
\right].
$$

Раскроем скобки:
$$
P = \frac{1}{4}\int_0^1 \eta^2\, d\eta
-\frac{1}{2}\int_0^1 \eta^2\ln\eta\, d\eta
+\frac{\ln 4}{4}\int_0^1 \eta^2\, d\eta.
$$

Соберём интегралы по одинаковым функциям:
$$
P
=
\left(\frac{1}{4} + \frac{\ln 4}{4}\right)\int_0^1 \eta^2\, d\eta
-\frac{1}{2}\int_0^1 \eta^2\ln\eta\, d\eta.
$$

Вычислим сначала
$$
\int_0^1 \eta^2\, d\eta = \left[\frac{\eta^3}{3}\right]_0^1 = \frac{1}{3}.
$$

Теперь вычислим
$$
\int_0^1 \eta^2\ln\eta\, d\eta.
$$

Интегрируем по частям:  
пусть $u = \ln\eta$, $dv = \eta^2 d\eta$. Тогда
$$
du = \frac{1}{\eta} d\eta,\qquad v = \frac{\eta^3}{3}.
$$

По формуле интегрирования по частям:
$$
\int_0^1 \eta^2\ln\eta\, d\eta
=
\left.\frac{\eta^3}{3}\ln\eta\right|_0^1
-
\int_0^1 \frac{\eta^3}{3}\cdot\frac{1}{\eta}\, d\eta
=
\left.\frac{\eta^3}{3}\ln\eta\right|_0^1
-
\frac{1}{3}\int_0^1 \eta^2\, d\eta.
$$

На верхнем пределе $\eta=1$: $\ln 1 = 0$, значит $\dfrac{\eta^3}{3}\ln\eta = 0$.  
На нижнем пределе $\eta\to 0^+$ выражение $\eta^3\ln\eta \to 0$, поэтому граничный член также равен 0.

Значит,
$$
\int_0^1 \eta^2\ln\eta\, d\eta
=
-\frac{1}{3}\int_0^1 \eta^2\, d\eta
=
-\frac{1}{3}\cdot\frac{1}{3}
=
-\frac{1}{9}.
$$

Подставляем оба результата в выражение для $P$:
$$
P
=
\left(\frac{1}{4} + \frac{\ln 4}{4}\right)\cdot\frac{1}{3}
-\frac{1}{2}\cdot\left(-\frac{1}{9}\right).
$$

Сначала упростим первый множитель:
$$
\left(\frac{1}{4} + \frac{\ln 4}{4}\right)\cdot\frac{1}{3}
=
\frac{1+\ln 4}{12}.
$$

Второй член:
$$
-\frac{1}{2}\cdot\left(-\frac{1}{9}\right)
=
\frac{1}{18}.
$$

Итак,
$$
P = \frac{1+\ln 4}{12} + \frac{1}{18}.
$$

Приведём к общему знаменателю $36$:
$$
\frac{1+\ln 4}{12} = \frac{3(1+\ln 4)}{36},\qquad
\frac{1}{18} = \frac{2}{36}.
$$

Тогда
$$
P = \frac{3(1+\ln 4)}{36} + \frac{2}{36}
=
\frac{3 + 3\ln 4 + 2}{36}
=
\frac{5 + 3\ln 4}{36}.
$$

Так как $\ln 4 = \ln(2^2) = 2\ln 2$, можно записать и так:
$$
P = \frac{5 + 6\ln 2}{36}.
$$

Численно:
$$
P \approx \frac{5 + 6\ln 2}{36} \approx 0.2544.
$$

Это и есть искомая вероятность того, что уравнение  
$\xi x^2 + \eta x + \zeta = 0$, где $(\xi,\eta,\zeta)$ равномерно распределены в $[0,1]^3$, имеет действительные корни.

**Python: аналитическое решение + Монте-Карло**

In [ ]:
import random
import math


def analytic_probability() -> float:
    """
    Аналитическое решение:
    P = (5 + 6 ln 2) / 36
    """
    return (5.0 + 6.0 * math.log(2.0)) / 36.0


def has_real_roots(xi: float, eta: float, zeta: float) -> bool:
    """
    Проверяем условие действительности корней:
    D = η^2 - 4 ξ ζ >= 0
    """
    D = eta * eta - 4.0 * xi * zeta
    return D >= 0.0


def monte_carlo_probability(n_trials: int = 100_000, seed: int = 42) -> float:
    """
    Оценка вероятности методом Монте-Карло:
    генерируем (ξ, η, ζ) ~ U([0,1]^3)
    и считаем долю случаев с D >= 0
    """
    rng = random.Random(seed)
    count_real = 0

    for _ in range(n_trials):
        xi = rng.random()
        eta = rng.random()
        zeta = rng.random()

        if has_real_roots(xi, eta, zeta):
            count_real += 1

    return count_real / n_trials


if __name__ == "__main__":
    p_an = analytic_probability()

    print("Аналитическое значение:")
    print(f"P = (5 + 6 ln 2) / 36 = {p_an:.6f}\n")

    trials_list = [10_000, 50_000, 100_000, 500_000]
    results = []

    for n in trials_list:
        p_mc = monte_carlo_probability(n_trials=n, seed=42)
        results.append((n, p_mc))

    # таблица
    h_line = ("-"*60)
    print(h_line)
    print(f"{'n_trials':>10} | {'Monte Carlo':>12} | {'Analytical':>12} | {'Delta':>12}")
    print(h_line)
    delta_list = []
    for n, p_mc in results:
        delta = abs(p_mc - p_an)
        print(f"{n:10d} | {p_mc:12.6f} | {p_an:12.6f} | {delta:12.6f}")
        delta_list.append(delta)
    print(h_line)
    
    # среднее значение Delta
    print(f"Среднее значение Delta: {sum(delta_list) / len(delta_list):.6f}")

Аналитическое значение:
P = (5 + 6 ln 2) / 36 = 0.254413

------------------------------------------------------------
  n_trials |  Monte Carlo |   Analytical |        Delta
------------------------------------------------------------
     10000 |     0.250500 |     0.254413 |     0.003913
     50000 |     0.254160 |     0.254413 |     0.000253
    100000 |     0.255770 |     0.254413 |     0.001357
    500000 |     0.254766 |     0.254413 |     0.000353
------------------------------------------------------------
Среднее значение Delta: 0.001469
